<h1>Majurca</h1>

In [1]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

<h2>Loading the data</h2>

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
#         rotation_range=20,
        brightness_range=[0.3,1.0],
        validation_split=0.1
)

batch_size = 64
shape = (224, 224)

base_dir = "/home/otiose/repos/epita/majurca/"

#data_dir = base_dir + "data"
data_dir = base_dir + "subset"

train_generator = train_datagen.flow_from_directory(
        directory=data_dir,
        target_size=shape,
        batch_size=batch_size,
        subset="training",
        class_mode="categorical")

validation_generator = train_datagen.flow_from_directory(
        directory=data_dir,
        target_size=shape,
        batch_size=batch_size,
        subset="validation",
        class_mode="categorical",
        shuffle=False
    )

Found 9413 images belonging to 15 classes.
Found 1043 images belonging to 15 classes.


In [5]:
from sklearn.utils import class_weight
import numpy as np

class_weights_arr = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes), 
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights_arr))

In [6]:
inputShape = (shape+(3,))
outputShape = 15

<h2>Creating the ANN</h2>

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import (
    Flatten, 
    Dense,
    GlobalAveragePooling2D, 
    Dropout
)
from tensorflow.keras.models import Model, Sequential

with mirrored_strategy.scope():
    base_model = VGG16(weights="imagenet", include_top=False)

    x = base_model.get_layer('block5_conv3').output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(outputShape, activation='sigmoid')(x)

    model = Model(inputs=base_model.inputs, outputs=x)

<h2>Training The ANN: Transfer Learning</h2>

<h3>Frozen weight pre-training</h3>

In [8]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import clone_model

In [9]:
with mirrored_strategy.scope():
    for layer in base_model.layers:
        layer.trainable = False

    optimizer = Adam()

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    class_weight=class_weights,
    epochs=10
)

Epoch 1/10
148/148 [==============================] - 116s 785ms/step - loss: 2.7094 - accuracy: 0.0717 - val_loss: 2.6777 - val_accuracy: 0.0729
Epoch 2/10
148/148 [==============================] - 53s 360ms/step - loss: 2.6390 - accuracy: 0.1219 - val_loss: 2.6029 - val_accuracy: 0.1352
Epoch 3/10
148/148 [==============================] - 53s 361ms/step - loss: 2.5691 - accuracy: 0.1481 - val_loss: 2.6586 - val_accuracy: 0.0729
Epoch 4/10
148/148 [==============================] - 53s 361ms/step - loss: 2.4739 - accuracy: 0.1626 - val_loss: 2.4746 - val_accuracy: 0.1659
Epoch 5/10
148/148 [==============================] - 53s 360ms/step - loss: 2.3848 - accuracy: 0.2043 - val_loss: 2.5599 - val_accuracy: 0.1352
Epoch 6/10
148/148 [==============================] - 54s 368ms/step - loss: 2.3160 - accuracy: 0.2185 - val_loss: 2.4622 - val_accuracy: 0.1534
Epoch 7/10
148/148 [==============================] - 55s 375ms/step - loss: 2.2490 - accuracy: 0.2419 - val_loss: 2.4778 - val_a

In [10]:
for layer in base_model.layers:
    layer.trainable = True
    
with mirrored_strategy.scope():
    optimizer = Adam(lr=1e-5)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

<h3>Loading previous model</h3>

In [11]:
# from tensorflow.keras.models import load_model

# model = load_model(base_dir + "model/model-02/model.h5")

<h3>Unfrozen full training</h3>

In [ ]:
history = model.fit(
    x=train_generator,
    validation_data=validation_generator,
    class_weight=class_weights,
    epochs=500
)

Epoch 1/500
148/148 [==============================] - 157s 1s/step - loss: 1.8837 - accuracy: 0.3420 - val_loss: 2.1992 - val_accuracy: 0.2589
Epoch 2/500
148/148 [==============================] - 100s 675ms/step - loss: 1.6318 - accuracy: 0.4177 - val_loss: 1.7574 - val_accuracy: 0.3816
Epoch 3/500
148/148 [==============================] - 102s 692ms/step - loss: 1.4591 - accuracy: 0.4733 - val_loss: 1.8047 - val_accuracy: 0.3826
Epoch 4/500
148/148 [==============================] - 105s 709ms/step - loss: 1.3408 - accuracy: 0.4977 - val_loss: 1.6154 - val_accuracy: 0.4554
Epoch 5/500
  1/148 [..............................] - ETA: 0s - loss: 1.0976 - accuracy: 0.5469

<h3>Saving trained model</h3>

In [ ]:
import os
import pandas as pd
from tensorflow.keras.models import save_model

model_dir = base_dir + "model/"

dirlist = os.listdir(model_dir)
if len(dirlist) == 0:
    iteration = "01"
else:
    last_iteration = int(dirlist[-1].split("-")[1])
    iteration = "{:02d}".format(last_iteration + 1)

save_dir = model_dir + "model-" + iteration + "/"
os.mkdir(save_dir)

save_model(model, save_dir + "model.h5", save_format="h5")
hist_df = pd.DataFrame(history.history) 
hist_csv_file = save_dir + "history.csv"

with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred_prob = model.predict(validation_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = validation_generator.classes

<h2>Performance analysis</h2>

In [ ]:
target_names = [os.path.basename(path) for path in glob(data_dir + "/*")]

print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
import seaborn as sn
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cm, index = labels, columns = labels)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,cmap=plt.cm.Blues)
plt.show()